In [12]:
import pandas as pd

In [13]:
data = pd.read_excel('/jdfssz1/ST_HEALTH/P20Z10200N0206/renzirui/Projects/GIZ_SouthernChinaBat_clean/BatPopGenetics/Rhinolophus_affinis/7.Fstcalc/TableS1.Samples information_2023-08-01.xlsx')
data

,Sample_name,sn,Host_species,Sample_type,Province,City,Site,Longitude,Latitude,Collection_time,Raw_Base(Gb),Clean_Base(Gb),Clean_Read1_num,Clean_Read1_avg_len,Clean_Read2_avg_len
0,B100,R2109032424,Rhinolophus affinis,Anal swab,Guangdong,Guangzhou,Site1_GZ,113.837428,23.737583,2020-04-29,30.40,14.670611,51000989,143.8,143.8
1,B101,R2109032425,Rhinolophus affinis,Anal swab,Guangdong,Guangzhou,Site1_GZ,113.837428,23.737583,2020-04-29,40.84,8.081463,27792230,145.4,145.4
2,B102,R2109032426,Rhinolophus affinis,Anal swab,Guangdong,Guangzhou,Site1_GZ,113.837428,23.737583,2020-04-29,44.26,19.282886,66686087,144.6,144.6
3,B103,R2109032427,Rhinolophus affinis,Anal swab,Guangdong,Guangzhou,Site1_GZ,113.837428,23.737583,2020-04-29,36.50,15.505530,53661587,144.5,144.5
4,B104,R2109032428,Rhinolophus affinis,Anal swab,Guangdong,Guangzhou,Site1_GZ,113.837428,23.737583,2020-04-29,54.46,21.030041,73139115,143.8,143.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
600,HL19,R2012010352,Pipistrellus abramus,Anal swab,Hainan,Qiongzhong,Site12_QZ,109.677432,18.997514,2008-07-03,48.26,9.266148,32886857,141.1,140.6
601,HL20,R2012010353,Pipistrellus abramus,Anal swab,Hainan,Qiongzhong,Site12_QZ,109.677432,18.997514,2008-07-03,52.42,10.586133,37188437,142.5,142.2
602,XL26,R2109032618,Hipposideros pomona,Anal swab,Hainan,Qiongzhong,Site13_QZ,109.751167,18.928437,2008-07-06,12.14,9.428727,33615176,140.3,140.2
603,XL31,R2109032619,Rhinolophus affinis,Anal swab,Hainan,Qiongzhong,Site13_QZ,109.751167,18.928437,2008-07-06,6.74,5.554121,20314893,136.7,136.7


In [14]:
data['Year'] = data['Collection_time'].apply(lambda x: x.year)

In [15]:
Ra_data = data.loc[(data['Host_species'] == 'Rhinolophus affinis') & (data['Site'].isin(['Site1_GZ', 'Site6_HZ'])), ['sn', 'Site', 'Year']]
Ra_data['Site_Year'] = Ra_data['Site'] + '_' + Ra_data['Year'].astype(str)
Ra_data

,sn,Site,Year,Site_Year
0,R2109032424,Site1_GZ,2020,Site1_GZ_2020
1,R2109032425,Site1_GZ,2020,Site1_GZ_2020
2,R2109032426,Site1_GZ,2020,Site1_GZ_2020
3,R2109032427,Site1_GZ,2020,Site1_GZ_2020
4,R2109032428,Site1_GZ,2020,Site1_GZ_2020
...,...,...,...,...
451,R2107024477,Site6_HZ,2013,Site6_HZ_2013
452,R2107024478,Site6_HZ,2013,Site6_HZ_2013
456,R2107024482,Site6_HZ,2013,Site6_HZ_2013
461,R2107024487,Site6_HZ,2013,Site6_HZ_2013


In [16]:
def runcommand(command):
    import subprocess
    p = subprocess.Popen(command, stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE, shell=True)
    stdout, stderr = p.communicate()
    code = p.returncode
    return(code, stdout, stderr)


In [17]:
vcf = '/jdfssz1/ST_HEALTH/P20Z10200N0206/renzirui/Projects/GIZ_SouthernChinaBat_clean/BatPopGenetics/Rhinolophus_affinis/3.vcf_merge_QC/3.6.filtsampsite.vcf'

In [18]:
for line in open(vcf).readlines():
	if line.startswith('#CHROM'):
		vcfdfheader = line.lstrip('#').split()
		break
vcf_samples = set(vcfdfheader[9:])

In [19]:
def fst_groupcalc(vcf, group2sn, outputdir='./'):
    from pathlib import Path
    Path(outputdir).mkdir(exist_ok = True)
    fst_result = []
    for i in range(len(group2sn)):
        pop1 = group2sn[i][0]
        open(f"{outputdir}/{pop1}.list", 'w').write('\n'.join(group2sn[i][1]))
        for j in range(i+1, len(group2sn)):
            pop2 = group2sn[j][0]
            open(f"{outputdir}/{pop2}.list", 'w').write('\n'.join(group2sn[j][1]))
            code, stdout, stderr = runcommand(f"vcftools --vcf {vcf} --weir-fst-pop {outputdir}/{pop1}.list --weir-fst-pop {outputdir}/{pop2}.list --out {outputdir}/{pop1}-{pop2}.fst")
            if code:
                print(code, stderr.decode())
                raise Exception("Non-zero exit when executing vcftools!")
            else:
                stderr_contentlist = stderr.decode().splitlines()
                #print(stderr_contentlist)
                fst = float(stderr_contentlist[-4].split()[-1])
                weighted_fst = float(stderr_contentlist[-3].split()[-1])
                pop1_usedsamp = len(set(group2sn[i][1]) & vcf_samples)
                pop2_usedsamp = len(set(group2sn[j][1]) & vcf_samples)
                fst_result.append([pop1, pop1_usedsamp, pop2, pop2_usedsamp, fst, weighted_fst])
    return pd.DataFrame(fst_result, columns=['pop1', 'pop1_sampnum', 'pop2', 'pop2_sampnum', 'Fst', 'weighted_Fst'])

In [20]:
with pd.ExcelWriter('Ra_Fst_result.xlsx') as writer:
	for groupvar in ['Site', 'Year', 'Site_Year']:
		group2sn = [ (site, list(dfsn['sn'])) for site, dfsn in Ra_data[['sn', groupvar]].groupby(groupvar) ]
		fstdf = fst_groupcalc(vcf, group2sn, groupvar)
		fstdf.to_excel(writer, sheet_name=groupvar, index=False)